# Schelling game

**Inspired by:** Schelling, T. C. (1980). _The Strategy of Conflict._ Harvard university press.

Alice and Bob agree to meet at a bar on Sunday, but they forget to decide on a bar to meet at. One bar is slightly more popular than the other. Where do they go? We can model this with recursive reasoning — Alice thinks about where Bob thinks Alice might go.

In [1]:
from memo import memo
import jax
import jax.numpy as np

Bar = np.arange(2)
@jax.jit
def prior(b): return np.array([0.55, 0.45])[b]

@memo
def alice[b: Bar](depth):
    alice: thinks[ bob: chooses(b in Bar, wpp=bob[b](depth - 1)) ]
    alice: chooses(b in Bar, wpp=prior(b) * Pr[b == bob.b])
    return Pr[alice.b == b]

@memo
def bob[b: Bar](depth):
    bob: thinks[ alice: chooses(b in Bar, wpp=alice[b](depth) if depth > 0 else 1) ]
    bob: chooses(b in Bar, wpp=prior(b) * Pr[b == alice.b])
    return Pr[bob.b == b]

for i in range(1, 10):
    print(f'alice({i}) = {alice(i)}; bob({i}) = {bob(i)}')

alice(1) = [0.59900993 0.40099007]; bob(1) = [0.64611655 0.35388345]
alice(2) = [0.6905481  0.30945188]; bob(2) = [0.73171747 0.26828253]
alice(3) = [0.76923996 0.23076005]; bob(3) = [0.8029279 0.1970721]
alice(4) = [0.832767   0.16723298]; bob(4) = [0.8588822 0.1411178]
alice(5) = [0.88149947 0.11850046]; bob(5) = [0.90091014 0.09908987]
alice(6) = [0.91743904 0.08256097]; bob(6) = [0.9314207  0.06857934]
alice(7) = [0.9431811  0.05681884]; bob(7) = [0.9530266  0.04697341]
alice(8) = [0.9612362  0.03876386]; bob(8) = [0.96805906 0.031941  ]
alice(9) = [0.9737139 0.0262862]; bob(9) = [0.97838986 0.02161017]


Notice the rapid convergence to the more popular bar.

How confident is Alice in meeting Bob? How confident is an observer that Alice will meet Bob? Which do you expect to be higher?

In [2]:
@memo
def alice_confidence(depth):
    alice: thinks[ bob: chooses(b in Bar, wpp=bob[b](depth - 1)) ]
    alice: chooses(b in Bar, wpp=prior(b) * Pr[b == bob.b])
    return E[alice[Pr[b == bob.b]]]

@memo
def obs_confidence(depth):
    alice: chooses(b in Bar, wpp=alice[b](depth))
    bob: chooses(b in Bar, wpp=bob[b](depth))
    return Pr[alice.b == bob.b]

for i in range(1, 10):
    print(f'Alice is {alice_confidence(i):.3f} confident, observer is {obs_confidence(i):.3f} confident.')

Alice is 0.510 confident, observer is 0.529 confident.
Alice is 0.556 confident, observer is 0.588 confident.
Alice is 0.625 confident, observer is 0.663 confident.
Alice is 0.702 confident, observer is 0.739 confident.
Alice is 0.774 confident, observer is 0.806 confident.
Alice is 0.835 confident, observer is 0.860 confident.
Alice is 0.882 confident, observer is 0.902 confident.
Alice is 0.918 confident, observer is 0.932 confident.
Alice is 0.943 confident, observer is 0.953 confident.


The observer is always slightly more confident, because they think both Alice and Bob are thinking at level $i$. Alice on the other hand thinks Bob is thinking at level $i-1$.

The code below reproduces the scaled-up experiment shown in the paper (100 bars, 100 levels of recursion).

In [3]:
Bar = np.arange(100)
@jax.jit
def prior(b): return 1

@memo
def alice[b: Bar](depth):
    alice: thinks[ bob: chooses(b in Bar, wpp=bob[b](depth - 1)) ]
    alice: chooses(b in Bar, wpp=prior(b) * Pr[b == bob.b])
    return Pr[alice.b == b]

@memo
def bob[b: Bar](depth):
    bob: thinks[ alice: chooses(b in Bar, wpp=alice[b](depth) if depth > 0 else 1) ]
    bob: chooses(b in Bar, wpp=prior(b) * Pr[b == alice.b])
    return Pr[bob.b == b]

alice(100)
%timeit -r 10 -n 100 alice(100).block_until_ready()

4.35 ms ± 44.8 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
